# Defi Wind - Results Download

Copyright 2024 Michael George (AKA Logiqx).

This file is part of [defi-results](https://github.com/Logiqx/defi-results) and is distributed under the terms of the GNU General Public License.

defi-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import sys
import json
import csv

import urllib.request

import unicodedata
import re

In [2]:
projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

In [3]:
CONFIG_DIR = 'config'

EVENTS_CONFIG = 'events.json'
COUNTRIES_CSV = 'countries.csv'

In [4]:
if __name__ == '__main__':
    # Load event config
    filename = os.path.join(projdir, CONFIG_DIR, EVENTS_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        events = json.loads(jsonTxt)

    # Load country codes
    countryCodes = {}
    filename = os.path.join(projdir, CONFIG_DIR, COUNTRIES_CSV)
    with open(filename, 'r', encoding='utf-8') as f:
        csvReader = csv.DictReader(f)
        for values in csvReader:
            countryCodes[values['Alpha-2 Code']] = values

In [5]:
def slugify(value):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

In [6]:
DATA_DIR = 'data'
JSON_DIR = 'json'
CSV_DIR = 'csv'

EVENT_NAME = 'name'
EVENT_FIELDS = 'fields'
EVENT_RACES = 'races'

RACE_NAME = 'name'
RACE_URL = 'url'

RESULTS_LIST = 'list'
RESULTS_FIELDS = 'Fields'
RESULTS_DATA = 'data'

FIELD_LABEL = 'Label'
FIELD_RACE_NO = 'RaceNo'
FIELD_PLACE = 'Place'
FIELD_TALLY = 'Tally'
FIELD_NAT = 'Nat'
FIELD_SEX = 'HelmSex'
FIELD_CLASS = 'Class'
FIELD_AGE_GROUP = 'HelmAgeGroup'
FIELD_POINTS = 'Points'

In [7]:
def getRaceFields(event, results):
    '''Get field mappings from results'''
    
    fieldMappings = {}

    # Nationality is sometimes blank
    labels = [field[FIELD_LABEL] or 'Nat.' for field in results[RESULTS_LIST][RESULTS_FIELDS]]

    # Determine index of all fields
    for field, aliases in event[EVENT_FIELDS].items():
        fieldMapping = None

        for alias in aliases:
            if alias in labels:
                fieldMapping = labels.index(alias)
                break
                
        fieldMappings[field] = fieldMapping

    return fieldMappings


def getRaceData(event, raceNo, results, fieldMappings):
    '''Get data from results'''
      
    data = []
    place = None

    for result in results[RESULTS_DATA]:
        # Defi Winf 2023 and Defi Wind 2022 (run 5) has bib number in the first field
        if result[0] != result[2].replace('#', ''):
            offset = 2
        else:
            offset = 1

        record = [raceNo]
        for fieldName, fieldMapping in fieldMappings.items():
            # Start with the raw field value
            if fieldMapping is not None:
                value = result[fieldMapping + offset]
            else:
                value = ''
            
            # Remove trailing puncuation from place
            if fieldName == FIELD_PLACE:
                value = re.sub("[^0-9]*$", "", value)

                # Handle oint results (blank places)
                value = value or place
                place = value

            # Remove leading puncuation from tally number
            elif fieldName == FIELD_TALLY:
                value = re.sub("^[^0-9]*", "", value)

            # Extract nationality from image name
            elif fieldName == FIELD_NAT:
                value = value.split('/')[-1]
                value = os.path.splitext(value)[0]
                value = re.sub("_.*", "", value).upper()
                value = countryCodes[value]['Name']

            # Remove placing for sex
            elif fieldName == FIELD_SEX:
                value = re.sub(" \(.*\)$", "", value)

            # Remove placing for class
            elif fieldName == FIELD_CLASS:
                value = re.sub(" \(.*\)$", "", value)
                
                # Category = fin / foil / open
                if value == 'Libre':
                    value = 'Open'
                elif value == 'Aileron':
                    value = 'Fin'

            # Remove placing for age group
            elif fieldName == FIELD_AGE_GROUP:
                value = re.sub(" \(.*\)$", "", value)

            # Replace 0,7 with 0.7
            elif fieldName == FIELD_POINTS:
                value = value.replace(',', '.')

            record.append(value)

        data.append(record)

    print(data[0])
    print()

    return data
        

def processEvent(event):
    '''Process single event'''

    eventSlug = slugify(event[EVENT_NAME])

    jsonPath = os.path.join(projdir, DATA_DIR, JSON_DIR, eventSlug)
    if not os.path.exists(jsonPath):
        os.makedirs(jsonPath)

    header = [FIELD_RACE_NO] + [field for field in event[EVENT_FIELDS]]
    csvData = [header]

    raceNo = 1
    for race in event[EVENT_RACES]:
        print(f'Downloading {event[EVENT_NAME]} - {race[RACE_NAME]}...')

        with urllib.request.urlopen(race[RACE_URL]) as f:
            jsonTxt = f.read().decode('utf-8')
            results = json.loads(jsonTxt)
            jsonTxt = json.dumps(results, indent=2)

        raceFields = getRaceFields(event, results)
        raceData = getRaceData(event, raceNo, results, raceFields)
        
        csvData += raceData
        raceNo += 1

        raceSlug = slugify(race[RACE_NAME])
        fn = os.path.join(jsonPath, raceSlug + '.json')       
        with open(fn, 'w') as f:
            f.write(jsonTxt)
            
    csvPath = os.path.join(projdir, DATA_DIR, CSV_DIR)
    if not os.path.exists(csvPath):
        os.makedirs(csvPath)

    fn = os.path.join(csvPath, eventSlug + '.csv')
    with open(fn, 'w') as f:
        csvWriter = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        csvWriter.writerows(csvData)


for event in events:
    processEvent(event)

print('All done!')

[1, '1', '192', 'ALBEAU, Antoine', 'France', 'M', '40-49M', '00:33:00', '0.7']

[2, '1', '465', 'GOYARD, Nicolas', 'France', 'M', '20-29M', '00:32:11', '0.7']

[3, '1', '465', 'GOYARD, Nicolas', 'France', 'M', '20-29M', '00:31:22', '0.7']

[4, '1', '465', 'GOYARD, Nicolas', 'France', 'M', '20-29M', '00:34:24', '0.7']

[5, '1', '465', 'GOYARD, Nicolas', 'France', 'M', '20-29M', '', '0.7']

[1, '1', '465', 'GOYARD, NICOLAS', 'France', 'M', 'Foil', '20-39M', '00:32:19']

[2, '1', '465', 'GOYARD, NICOLAS', 'France', 'M', 'Foil', '20-39M', '00:30:35']

[3, '1', '465', 'GOYARD, NICOLAS', 'France', 'M', 'Foil', '20-39M', '00:30:50']

[4, '1', '37', 'SØE, JOHAN', 'Denmark', 'M', 'Open', 'U20M', '00:31:18']

All done!
